In [1]:
import pandas as pd
import sqlite3

<h2>Подключение к БД</h2>

In [2]:
conn = sqlite3.connect('../data/checking-logs.sqlite')
conn

<h2>Используя только один запрос для каждой из групп, создайте два фрейма данных: test_results и control_results со столбцами time и avg_diff и только двумя строками</h2>

- время должно иметь значения: после и до
- avg_diff содержит среднюю дельту для всех пользователей за период до того, как каждый из них впервые посетил страницу, и после
- учитывайте только тех пользователей, у которых есть наблюдения до и после

In [3]:
query = '''
SELECT time,
       avg(diff) AS avg_diff
FROM (SELECT uid,
             CAST( (JulianDay(datetime(deadlines.deadlines, 'unixepoch')) -
                    JulianDay(test.first_commit_ts)
                    ) * 24 as Integer
                  ) as diff,
             CASE WHEN test.first_commit_ts < test.first_view_ts THEN 'before'
             ELSE 'after' END AS time
       FROM test left join deadlines on test.labname = deadlines.labs
       WHERE labname <> 'project1'
      )
WHERE uid in (SELECT uid
              FROM (SELECT uid,
                           CASE WHEN test.first_commit_ts < test.first_view_ts THEN 'before'
                           ELSE 'after' END as time
                    FROM test
                    LEFT JOIN deadlines ON test.labname=deadlines.labs
                    WHERE labname <> 'project1'
                    )
               GROUP BY uid
               HAVING COUNT(DISTINCT time)=2
               )
GROUP BY time
'''
test_results = pd.io.sql.read_sql(query, conn)
test_results

,time,avg_diff
0,after,104.6000
1,before,60.5625


In [4]:
query = '''
SELECT time,
       avg(diff) AS avg_diff
FROM (SELECT uid,
             CAST( (JulianDay(datetime(deadlines.deadlines, 'unixepoch')) -
                    JulianDay(control.first_commit_ts)
                    ) * 24 as Integer
                  ) as diff,
             CASE WHEN control.first_commit_ts < control.first_view_ts THEN 'before'
             ELSE 'after'
             END AS time
             FROM control
             LEFT JOIN deadlines ON control.labname=deadlines.labs
             WHERE labname <> 'project1'
      )
WHERE uid IN (SELECT uid
              FROM (SELECT uid,
                           CASE WHEN control.first_commit_ts < control.first_view_ts THEN 'before'
                           ELSE 'after'
                           END AS time
                    FROM control
                    LEFT JOIN deadlines ON control.labname=deadlines.labs
                    WHERE labname <> 'project1'
                    )
              GROUP BY uid
              HAVING COUNT(distinct time)=2
              )
GROUP BY time
'''
control_results = pd.io.sql.read_sql(query, conn)
control_results

,time,avg_diff
0,after,117.636364
1,before,99.464286


<h2>Закрытие соединения</h2>

In [5]:
conn.close()